# 什么是 XGBoost
- `XGBoost` 是处理标准表格数据(`DataFrame`类型数据)的重要模型。

- 为了达到较高的准确率，XGBoost比其他模型需要更多的知识和模型调参。
- `XGBoost` 实际上是 GBDT(Gradient Boosted Decision Tree) 算法的一种实现。

- 从一个图开始，简单了解 GBDT。

    ![](img/002.png)
    - 通过循环，不停地创建新的模型，并将其并入模型集。通过计算数据集中每个观测值的误差来开始周期，然后建立一个新模型来预测这些模型。我们将这个误差预测模型的预测添加到“模型集合”中。
    - 为了进行预测，我们添加了以前所有模型的预测。我们可以使用这些预测来计算新错误，建立下一个模型，并将其添加到模型集中。
    - 那个周期以外还有一个简单模型(Naive Model)。我们需要一些基础预测来启动周期。在实践中，最初的预测可能非常不准确，随后对模型集的补充将解决这些错误。
    
# 示例
首先将数据集预加载为 `train_X, test_X, train_y, test_y`。

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer

data = pd.read_csv('input/train.csv')
data.dropna(axis=0, subset=['SalePrice'], inplace=True)

y = data.SalePrice
X = data.drop(['SalePrice'], axis=1).select_dtypes(exclude=['object'])

train_X, test_X, train_y, test_y = train_test_split(X.as_matrix(), y.as_matrix(), test_size=0.25)

my_imputer = Imputer()
train_X = my_imputer.fit_transform(train_X)
test_X = my_imputer.transform(test_X)

In [3]:
print(my_imputer.get_params())

{'axis': 0, 'copy': True, 'missing_values': 'NaN', 'strategy': 'mean', 'verbose': 0}


### 创建并训练模型

In [4]:
from xgboost import XGBRegressor

my_model = XGBRegressor()
my_model.fit(train_X, train_y, verbose=False)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

### 评估预测

In [5]:
predictions = my_model.predict(test_X)

from sklearn.metrics import mean_absolute_error
print("Mean Absolute Error : " + str(mean_absolute_error(predictions, test_y)))

Mean Absolute Error : 17932.0092145


### 模型调整
XGBoost有很少的参数可以影响模型准确率和训练速度。
#### `n_estimators` 及 `early_stopping_rounds`
`n_estimators` 指定上述模型圈循环次数。

![](img/003.png)
在欠拟合和过拟合比较图中，`n_estimators` 使得进一步向右移动。如果值太小，会出现欠拟合情况，这是训练集合测试集的错误率都很高；如果值太大，会出现过拟合的情况，这是在训练集中模型表现很好，但是在测试集中表现较差。可以使用训练数据试验获得理想的值。尽管`n_estimators`的取值和学习率相关，但是通常的取值范围为 100-1000。

`early_stopping_rounds` 参数提供了一种自动获取理想值得方法。提前停止导致模型迭代在验证分数不在改善时停止。一种明智的做法时，设置较大的`n_estimators`值，然后使用`early_stopping_rounds`找到合适的时间停止迭代。

由于随机因素，导致在某个迭代中，验证分数(validation score)没有改善，需要指定一个数字，以便在停止之前允许多少次这种退化情况。`early_stopping_rounds = 5`是一个很合理的取值。

下面是使用早停止的代码。

In [6]:
my_model = XGBRegressor(n_estimators=1000)
my_model.fit(train_X, train_y, early_stopping_rounds=5, eval_set=[(test_X, test_y)], verbose=False)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

当使用`early_stopping_rounds`时，需要留出一些数据来检查要使用的回合数。如果以后想要将模型与所有数据相匹配，将`n_estimators`设置为在早期停止运行时发现的最佳值。

### `learning_rate`
通常一个小的学习速率以及一个大的 `estimators`，会产生更精确的 XGBoost 模型，但是需要更多的训练时间。设置学习速率的代码如下。

In [7]:
my_model = XGBRegressor(n_estimators=1000, learning_rate=0.05)
my_model.fit(train_X, train_y, early_stopping_rounds=5, eval_set=[(test_X, test_y)], verbose=False)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.05, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

### `n_jobs`
在处理大的数据集的时候，往往需要考虑运行时间。可以利用并行计算，来加快模型构建。通常通过设置 `n_jobs` 为机器的CPU核心数，来实现并行计算。